### Import all the libraries and load the dfs needed

In [11]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [13]:
import os
path = os.chdir('C:/Users/jewel.espiritu/OneDrive - AMDATEX LAS PINAS SERVICES INC/Python/py')

### Merge all excel sheet into one Dataframe

In [14]:
sladf = pd.concat(pd.read_excel('./data/WeeklySLA_Data_2022.xlsx', sheet_name=None), ignore_index=True)


### Create a pivot table and exclude tickets older than 14 days

In [15]:
SLApivot = pd.pivot_table(sladf, index=['Service', 'Olderthan14', 'Week'], columns=['TTR Result\n(Excluding Pending Time)'], values=['Ticket No'], aggfunc='count')
paretopvt = SLApivot.query('Olderthan14=="N"')

### drop the extra columns of the pivot

In [16]:
paretopvt.columns = paretopvt.columns.droplevel(0)
paretopvt.columns.name = None
newdf = paretopvt.reset_index()


### Turn NaN values into 0 and Remove decimal using lambda

In [17]:
newdf['Beyond TAT'] = newdf['Beyond TAT'].fillna(0)
newdf['Within TAT'] = newdf['Within TAT'].fillna(0)

newdf['Beyond TAT'] = newdf['Beyond TAT'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['Within TAT'] = newdf['Within TAT'].astype(str).apply(lambda x: x.replace('.0',''))

newdf

,Service,Olderthan14,Week,Beyond TAT,Within TAT
0,"3rd Party/ Client Controlled Software, Network...",N,1,9,173
1,"3rd Party/ Client Controlled Software, Network...",N,2,35,226
2,"3rd Party/ Client Controlled Software, Network...",N,3,13,83
3,"3rd Party/ Client Controlled Software, Network...",N,4,9,161
4,"3rd Party/ Client Controlled Software, Network...",N,5,27,102
...,...,...,...,...,...
163,WFH PC setup,N,6,11,43
164,WFH PC setup,N,7,10,45
165,WFH PC setup,N,8,6,29
166,WFH PC setup,N,9,13,42


### Filter the df into current week only

In [18]:
newdf = newdf.query('Week == 10')
newdf

,Service,Olderthan14,Week,Beyond TAT,Within TAT
9,"3rd Party/ Client Controlled Software, Network...",N,10,18,139
19,Access,N,10,36,215
29,Email,N,10,11,55
39,Hardware,N,10,30,151
49,"IT software assessment, evaluation and testing",N,10,0,6
57,IT software transferring,N,10,2,6
66,MAJOR Incident (BSD System/Application),N,10,1,0
76,MINOR Incident (BSD System/Application),N,10,1,1
86,Network or internet,N,10,2,14
96,Onboarding Employee's IT-related Access and Ha...,N,10,0,19


### Add percentage column for beyond tat

In [19]:
sumbeyondtat = newdf['Beyond TAT'].astype(int).sum()
beyondpercent = newdf['Beyond TAT'].astype(int)/sumbeyondtat
newdf["Beyond TAT Percent"] = beyondpercent

C:\Users\JEWEL~1.ESP\AppData\Local\Temp/ipykernel_19920/1893037449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf["Beyond TAT Percent"] = beyondpercent


In [20]:
newdf = newdf.rename( columns = {'Beyond TAT': 'Beyond_TAT'}, inplace = False)
newdf.Beyond_TAT = newdf.Beyond_TAT.astype(float)
beyond = "Beyond_TAT"
newdf = newdf.sort_values(beyond, ascending=False)


In [21]:
newdf['Beyond_TAT'] = newdf['Beyond_TAT'].astype(str).apply(lambda x: x.replace('.0',''))
newdf['cumulative_sum'] = newdf[beyond].astype(int).cumsum()
newdf['cumulative_perc'] = 100*newdf['cumulative_sum']/newdf[beyond].astype(int).sum()
newdf

,Service,Olderthan14,Week,Beyond_TAT,Within TAT,Beyond TAT Percent,cumulative_sum,cumulative_perc
19,Access,N,10,36,215,0.260870,36,26.086957
39,Hardware,N,10,30,151,0.217391,66,47.826087
147,Software,N,10,23,117,0.166667,89,64.492754
9,"3rd Party/ Client Controlled Software, Network...",N,10,18,139,0.130435,107,77.536232
167,WFH PC setup,N,10,12,46,0.086957,119,86.231884
29,Email,N,10,11,55,0.079710,130,94.202899
57,IT software transferring,N,10,2,6,0.014493,132,95.652174
86,Network or internet,N,10,2,14,0.014493,134,97.101449
66,MAJOR Incident (BSD System/Application),N,10,1,0,0.007246,135,97.826087
76,MINOR Incident (BSD System/Application),N,10,1,1,0.007246,136,98.550725


## Create Pareto

In [22]:
subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig = px.bar(newdf, x="Service", y="Beyond TAT Percent", text="Beyond_TAT", color=newdf['Service'])
#fig.update_traces(texttemplate='%{text:.2%}', textposition='outside') #for percentage text
fig.update_traces(textposition='outside') #for absolute number text
for data in fig.data:
    data["width"] = 1

fig2 = px.line(newdf, x="Service", y="cumulative_perc")
fig2.update_traces(yaxis="y2", line=dict(color="Black", width=3))

subfig.add_traces(fig.data + fig2.data)
subfig.update_layout(autosize=False, width=1500, height=650, margin=dict(l=25, r=70, t=90, b=30), xaxis={'visible': False, 'showticklabels': False}, title="Beyond TAT Distribution")
subfig.show()